In [1]:
from py_vollib.black_scholes import black_scholes
from py_vollib.black_scholes.implied_volatility import implied_volatility
from py_vollib.black_scholes.greeks import analytical

import numpy as np 

import plotly.graph_objects as go


In [2]:
#class for option
class Option:
    def __init__(self, spot_price, strike_price, expiration, interest_rate, volatility, option_type,position):
        self._spot_price = spot_price
        self._strike_price = strike_price
        self._expiration = expiration
        self._interest_rate = interest_rate
        self._volatility = volatility
        self._option_type = option_type
        self._position = position


    @property
    def spot_price(self):
        return self._spot_price

    @property
    def strike_price(self):
        return self._strike_price

    @property
    def expiration(self):
        return self._expiration

    @property
    def interest_rate(self):
        return self._interest_rate

    @property
    def volatility(self):
        return self._volatility

    @property
    def option_type(self):
        return self._option_type
    
    @property
    def position(self):
        return self._position

    def price(self):
        """Calculate price by Black-Scholes model"""
        return black_scholes(self.option_type, self.spot_price, self.strike_price,
                             self.expiration/365, self.interest_rate, self.volatility)

    def delta(self):
        """Calculate delta"""
        return analytical.delta(self.option_type, self.spot_price, self.strike_price,
                                self.expiration/365, self.interest_rate, self.volatility)

    def vega(self):
        """Calculate vega"""
        return analytical.vega(self.option_type, self.spot_price, self.strike_price,
                               self.expiration/365, self.interest_rate, self.volatility)

    def theta(self):
        """Calculate theta"""
        return analytical.theta(self.option_type, self.spot_price, self.strike_price,
                                self.expiration/365, self.interest_rate, self.volatility)

    def gamma(self):
        """Calculate gama"""
        return analytical.gamma(self.option_type, self.spot_price, self.strike_price,
                                self.expiration/365, self.interest_rate, self.volatility)

    def rho(self):
        """Calculate rho"""
        return analytical.rho(self.option_type, self.spot_price, self.strike_price,
                              self.expiration/365, self.interest_rate, self.volatility)
    
    def _long_or_short(self):
        sign = 1 if self._position == 's' else -1
        return sign
    
    
    def _grid_price(self,spot,expiration):
        return black_scholes(self.option_type, spot, self.strike_price,
                             expiration/365, self.interest_rate, self.volatility)
    
    def generate_price_grid(self, min_x, max_x):
        
        spot_values = np.arange(min_x, max_x + 1, 1)
        expiration_values = np.arange(self.expiration, -1, -1)  # Expirace klesá od aktuální až k nule

        price_grid = np.zeros((len(expiration_values), len(spot_values)))

        for i, spot in enumerate(spot_values):
            for j, expiration in enumerate(expiration_values):
                price_grid[j, i] = round(self._grid_price(spot,expiration),3)

        sign = self._long_or_short()

        
        return  {"spot_values": spot_values, "expiration_values": expiration_values, "price_grid": (price_grid * sign)}


In [3]:
#Test result price - 4.51  USD
test_option = Option(142, 140, 25, 0.03, 0.22, "c",'l')
print(f"Call option price: {round(test_option.price(),2)} USD")

Call option price: 4.51 USD


In [4]:
option_1 = Option(142, 140, 25 , 0.03, 0.22, "c","s")
option_2 = Option(142, 140, 90 , 0.03, 0.22, "c","l")

In [6]:
option_1_grid = option_1.generate_price_grid(130, 150)

# Create the 3D surface plot
fig = go.Figure(data=go.Surface(
    x=option_1_grid['spot_values'],  
    y=option_1_grid['expiration_values'],
    z=option_1_grid['price_grid'],  
    colorscale='Viridis',
    colorbar=dict(title='Option Price'),
    hoverinfo='x+y+z',  # Zobrazení souřadnic a hodnoty při hoveru
    hovertemplate='Spot Price: %{x}<br>Expiration: %{y} days<br>Price: %{z}<extra></extra>'
))

# Update layout with titles and axis labels
fig.update_layout(
    title='3D Option Price Surface',
    scene=dict(
        xaxis_title='Spot Price',
        yaxis_title='Expiration (Days)',
        zaxis_title='Option Price'
    ),
    autosize=True
)

# Show the plot
fig.show()